In [1]:
import numpy as np 
import jVMC
import jax.numpy as jnp
import jax.random as jrnd
import matplotlib.pyplot as plt
from tqdm import tqdm



ebDim = 16
dep = 2
nH = 4
L = 3
N=3
ldim = 4
homFock = jnp.ones((1,1,L),dtype=int)

#net = jVMC.nets.RpxRWKV(L,LocalHilDim=ldim,hidden_size=hidden_size,num_heads=num_heads,embedding_size=emb_RWKV,num_layers=depth_RWKV,)
net = jVMC.nets.bosons.gpt_stupid.GPT(L,ldim,embeddingDim=ebDim,depth=dep,nHeads=nH)
seed = 252
key2 = jrnd.PRNGKey(seed)
numSamp = 2**0

psi_net= jVMC.vqs.NQS(net,seed=seed,batchSize=1)


In [2]:

sampler_net = jVMC.sampler.MCSampler(psi_net,(L,),key2)



In [3]:
psi_net(homFock),psi_net(homFock.at[1].set(0))


(Array([[-0.66630627]], dtype=float64), Array([[-0.66630627]], dtype=float64))

In [4]:
para = net.init(key2,homFock[0,0])


In [5]:
net.apply(para,jnp.expand_dims(0,0),block_states=None,output_state=True)

(Array([-3.54589369, -0.65656573, -0.79767378, -6.14119376], dtype=float64),
 (Array([0, 0, 0], dtype=int64), 1))

In [6]:

state = (jnp.full(L,-2,dtype=int), 1)
logit,new_state = net.apply(para,jnp.expand_dims(-1,0),block_states=state,output_state=True)
logit

Array([-0.93238777, -1.38925503, -1.16184274, -3.11863917], dtype=float64)

In [7]:
new_state

(Array([-2, -1, -2], dtype=int64), 2)

In [8]:
logit,new_state = net.apply(para,jnp.expand_dims(3,0),block_states=new_state,output_state=True)


In [9]:
psi_net(np.full((1,1,L),-2,dtype=int))

Array([[-1.70802146]], dtype=float64)

In [10]:
s,logit,p = sampler_net.sample(numSamples=10)

In [11]:
psi_net(s),logit

(Array([[-2.22200171, -2.02806914, -1.70802146, -1.22953871, -2.48653136,
         -1.46383455, -0.66630627, -1.96847104, -1.19598668, -1.19598668]],      dtype=float64),
 Array([[-2.22200171, -2.02806914, -1.70802146, -1.22953871, -2.48653136,
         -1.46383455, -0.66630627, -1.96847104, -1.19598668, -1.19598668]],      dtype=float64))